In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset
class TransformerModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
#         self.decoder = nn.Linear(ninp, ntoken)
        self.decoder = nn.Linear( ninp,ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
#         src = self.encoder(src) * math.sqrt(self.ninp)
        src = src * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
#        print(output.size())
        output = self.decoder(output)
        return output

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe=pe.to(device)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

def data_process(raw_text_iter):
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

In [3]:
# load array
Data = load('Data.npy')
# print the array

In [4]:
Data=np.nan_to_num(Data)

In [5]:
Xdata=[]
Ydata=[]
for idx in range(len(Data)//39):
    Xdata.append(Data[39*idx:39*(idx)+38][:,:].tolist())
    Ydata.append(Data[39*idx:39*(idx)+38][:,5].tolist())
Xdata=np.vstack(Xdata)
Ydata=np.vstack(Ydata)
Xdata=torch.FloatTensor(Xdata)
Ydata=torch.IntTensor(Ydata)
Ydata=Ydata.view(-1)

In [6]:
Ydata

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [7]:
Ydata=2*(Ydata>0).long()+(Ydata==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).double()

In [8]:
Ydata=Ydata.T

In [9]:
from torch.utils.data import DataLoader, random_split
Xtrain_data,Xtest_data=random_split(Xdata, (10, 15770))
Ytrain_data,Ytest_data=random_split(Ydata, (157700-15770, 15770))

Xtrain_data=Xtrain_data.dataset.data
Xtest_data=Xtest_data.dataset.data
Ytrain_data=Ytrain_data.dataset.data.long()
Ytest_data=Ytest_data.dataset.data.long()


In [10]:
Xtrain_data.size()

torch.Size([157700, 6])

In [11]:
Xtrain_data.size()

torch.Size([157700, 6])

In [12]:
Ytrain_data.size()

torch.Size([157700])

In [13]:
Xtrain_data.size()

torch.Size([157700, 6])

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

# batch_size = 25
# eval_batch_size = 25

# Xtrain_data = batchify(Xtrain_data, batch_size)
# Xtest_data = batchify(Xtest_data, eval_batch_size)
# Ytrain_data = batchify(Ytrain_data, batch_size)
# Ytest_data = batchify(Ytest_data, eval_batch_size)

In [15]:
bptt = 38
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [16]:
## ntokens = len(vocab) # 단어 사전(어휘집)의 크기
ntokens = 3 # 단어 사전(어휘집)의 크기
emsize =6 # 임베딩 차원
nhid = 200 # nn.TransformerEncoder 에서 피드포워드 네트워크(feedforward network) 모델의 차원
nlayers = 2 # nn.TransformerEncoder 내부의 nn.TransformerEncoderLayer 개수
nhead = 2 # 멀티헤드 어텐션(multi-head attention) 모델의 헤드 개수
dropout = 0.2 # 드랍아웃(dropout) 값
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [17]:
import time
A=[]
criterion = nn.CrossEntropyLoss()
lr = 50 # 학습률
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train():
    model.train() # 학습 모드를 시작합니다.
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, bptt)):

        data, org_targets = get_batch(Xtrain_data, i)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
        _,targets = get_batch(Ytrain_data,i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)

        print("data size",data.size())
        print("output size",output.size())
        print(output.view(-1, ntokens).size())
        print(org_targets.size())
        break
        targets=targets.repeat(38)
#        print(targets.size())
#         targets=targets.view(-1,ntokens)
#        print(targets.size())
        if(output.view(-1, ntokens).size()[0]!=targets.size()[0]):
            continue
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
#        print(loss.isnan())
        if(loss.isnan()):
            print(loss)
            print(targets)
            print("output is nan",output.isnan().any())
            print(output)
            A.append(output)
            A.append(data)
            break
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.3)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(Xtrain_data) // bptt, scheduler.get_last_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, Xdata_source,Ydata_source):
    eval_model.eval() # 평가 모드를 시작합니다.
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)

    with torch.no_grad():
        for i in range(0, Xdata_source.size(0) - 1, bptt):
            data, org_targets = get_batch(Xdata_source, i)
            _,targets = get_batch(Ydata_source,i)
            if (data.isnan().any() or data.isinf().any()):
#            print(data)
                continue
            targets=targets.repeat(38)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            if(output.view(-1, ntokens).size()[0]!=targets.size()[0]):
                continue
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(Xdata_source) - 1)

In [18]:
def evaluate_accuracy(eval_model, Xdata_source,Ydata_source):
    eval_model.eval() # 평가 모드를 시작합니다.
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(0, Xdata_source.size(0) - 1, bptt):
            data, org_targets = get_batch(Xdata_source, i)
            _,targets = get_batch(Ydata_source,i)
            if (data.isnan().any() or data.isinf().any()):
#            print(data)
                continue
            targets=targets.repeat(38)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            if(output.view(-1, ntokens).size()[0]!=targets.size()[0]):
                continue
            output_flat = output.view(-1, ntokens)
            _,predicted=torch.max(output_flat,1)
            total=len(Ydata_source)*38
            correct += (predicted == targets).sum().item()
            total_loss += len(data) * criterion(output_flat, targets).item()
    print(correct/total*100)
    return total_loss / (len(Xdata_source) - 1)

In [19]:
Xtrain_data.size()

torch.Size([157700, 6])

In [20]:
Ytrain_data.size()

torch.Size([157700])

In [21]:
Ytrain_data[0]

tensor(1)

In [28]:
sz=5

In [31]:
torch.triu(torch.ones((sz, sz)) == 1).transpose(0, 1)

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])

In [22]:
best_val_loss = float("inf")
epochs = 1 # 에포크 수
best_model = None
Xtrain_data=Xtrain_data.to(device)
Ytrain_data=Ytrain_data.to(device)
Xtest_data=Xtrain_data.to(device)
Ytest_data=Ytrain_data.to(device)
model=model.to(device)
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()

    val_loss = evaluate(model, Xtest_data,Ytest_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

torch.Size([38, 38, 6])
data size torch.Size([38, 6])
output size torch.Size([38, 38, 3])
torch.Size([1444, 3])
torch.Size([228])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

...

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

/root/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [23]:
PATH='best_model'
torch.save(best_model.state_dict(), PATH)

In [24]:
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [25]:
val_loss = evaluate_accuracy(model, Xtest_data,Ytest_data)
print('-' * 89)
print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
      'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                 val_loss, math.exp(val_loss)))
print('-' * 89)

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

...

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

In [26]:
best_val_loss

1.0768823518477229

In [27]:
val_loss = evaluate(model, Xtest_data,Ytest_data)

torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 38, 6])
torch.Size([38, 

KeyboardInterrupt: 

In [ ]:
output=A[0]

In [ ]:
data=A[1]

In [ ]:
data

In [ ]:
output[1].isnan().any()

In [ ]:
torch.cuda.is_available()